### Tests

In [ ]:
import sys, os
sys.path.append('../../src')

import astropy.coordinates as coord

from astropy.io.votable import parse
from astropy.table import Table
from astropy import units as u

import matplotlib.pyplot as plt
from pylab import rcParams
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd

from math import ceil
import gaia_utils as gu
from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
from astropy.utils.exceptions import AstropyWarning
warnings.simplefilter('ignore', category=AstropyWarning)

%matplotlib inline

## directory
rootdir = "/home/stephane/Science/cluster/GAIA"
wdir    = "%s/products"%(rootdir)
datadir = "%s/master/notebooks/data"%(rootdir)

os.chdir(wdir)

clustername = "NGC 2516"
RADIUS   = 3.0
kCluster = 9
WEIGHTVEL = 5.
votable_disk = False
BINSIZE = 200
SIGMA = 3.0
distclust = 373.0

In [ ]:
from astroquery.gaia import Gaia

# tables = Gaia.load_tables(only_names=True)

#for table in (tables):
#    print (table.get_qualified_name())
    

In [ ]:
# c = coord.SkyCoord.from_name("M47")
# c = coord.SkyCoord.from_name("NGC 2682")

print("## Query to GAIA...")
c = coord.SkyCoord.from_name(clustername)

queryaql = "SELECT * FROM gaiadr2.gaia_source WHERE \
CONTAINS(POINT('ICRS',gaiadr2.gaia_source.ra,gaiadr2.gaia_source.dec),CIRCLE('ICRS' \
,%f ,%f,%f))=1;"%(c.ra.deg+RADIUS/10.,c.dec.deg+RADIUS/10.,RADIUS)


# job = Gaia.launch_job_async( queryaql, dump_to_file=True)
# print (job)

print("## Query done...")
# data = job.get_results()

voname = "n2516.vot"
votable = parse(voname)

for table in votable.iter_tables():
    data = table.array
    print(data.dtype.names)

print("## data loaded....")

In [ ]:
## Filter the data
##

def plot_check(a1, a2, ifinal):
    "check the filltering"
    
    
    rcParams['figure.figsize'] = 14, 14
    f, axarr = plt.subplots(2, 2)

# ax.scatter(cluster_candidates["ra"], cluster_candidates["dec"], s=1, c="#000000")
    print(len(a1))
    print(len(a2))
    
    axarr[0,0].scatter(a1, a2, s=1, c="#000000")
    axarr[0,0].set_xlabel("a1")
    axarr[0,0].set_ylabel("a2")
    axarr[0,0].set_ylim([0,2000])

    axarr[0,1].scatter(a1[ifinal], a2[ifinal], s=1, c="#000000")
    axarr[0,1].set_xlabel("a1[ifinal]")
    axarr[0,1].set_ylabel("a2[ifinal]")

    plt.show()

def filter_data(lgal, bgal, distance, vra, vdec, cartesian = False, dist_range = [0., 2000], vra_range = [-200,200], vdec_range = [-200.,200]):
    "filter the data applying the range and return the sklearn-centric array"
    
    i1 = np.where((distance >= dist_range[0]) & (distance < dist_range[1]))
    i2 = np.where((vra >= vra_range[0]*WEIGHTVEL) & (vra < vra_range[1]*WEIGHTVEL))
    i12 = np.intersect1d(i1,i2)
    i3 = np.where((vdec >= vdec_range[0]*WEIGHTVEL) & (vdec < vdec_range[1]*WEIGHTVEL))
    ifinal = np.intersect1d(i12,i3)
    
    # plot_check(lgal,vra,ifinal)
    
    datask = np.zeros((len(ifinal),5))
    
    if cartesian:
        xx, yy, zz = convert_to_cartesian(lgal[ifinal], bgal[ifinal], distance[ifinal])
        datask[:,0] = xx
        datask[:,1] = yy
        datask[:,2] = zz
        datask[:,3] = vra[ifinal]
        datask[:,4] = vdec[ifinal]
    else:
        datask[:,0] = lgal[ifinal]
        datask[:,1] = bgal[ifinal]
        datask[:,2] = distance[ifinal]
        datask[:,3] = vra[ifinal]
        datask[:,4] = vdec[ifinal]
    
    return(datask)
    
    

In [ ]:
#Normalized the 5d datask with linear projection from [min,max] to [0,1]
def normalization0_1(data):
    result = np.zeros(data.shape)
    normalization_vector = np.zeros((5,2)) #Represente max and min    
    
    for i in range(5) :
        normalization_vector[i,0] = np.max(data[:,i]) # max
        normalization_vector[i,1] = np.min(data[:,i]) # min
        result[:,i] = (data[:,i]-normalization_vector[i,1])/(normalization_vector[i,0]-normalization_vector[i,1])  
        
    return(result,normalization_vector)

#Unnormalized Data with linear projection from [0,1] to [min,max]
def unnormalization0_1(data,normalization_vector):
    result = np.zeros(data.shape) 
    for i in range(5) :
        result[:,i] = data[:,i]*(normalization_vector[i,0]-normalization_vector[i,1]) + normalization_vector[i,1]
        
    return result

In [ ]:
## plot2D and plot3D


def plot2d_labels(a1,a2,labels,nclusterk,centroid,xylab = ["a1","b1"],title = "Title", clustername= "NGC"):
    figname = "%s-2Dkmeans-gaia.png"%(clustername)
    rcParams['figure.figsize'] = 14, 21
    nrow = int(ceil(nclust / 3))
    ncol = 3           
    f, axarr = plt.subplots(nrow, ncol)

    for i in range(nclusterk):
        ilabel = np.where(labels == i)[0]
        row = int(ceil(i / 3)) - 1
        col = i % 3
        axarr[row,col].scatter(a1[ilabel],a2[ilabel], s=1, c="#000000")
        axarr[row,col].set_xlabel(xylab[0])
        axarr[row,col].set_ylabel(xylab[1])
        
        txt = "Stars:%d Dist:%3.1f Vra:%3.1f Vdec:%3.1f"%(len(ilabel), centroid[i][2],centroid[i][3],centroid[i][4])
        axarr[row,col].text(-0.1,1.02, txt, size=12, ha="left", 
         transform=axarr[row,col].transAxes)
        
    plt.savefig(figname)
    plt.show()

def plot2d_filtered_labels(a1,a2,labels,nclusterk,centroid, xylab = ["a1","b1"],title = "Title", clustername= "NGC"):
    
    "filtering with wavelet."
    
    gaia = gu.gaiaSet()
    
    figname = "%s-2Dkmeans-filtered-gaia.png"%(clustername)
    rcParams['figure.figsize'] = 14, 21
    nrow = int(ceil(nclust / 3))
    ncol = 3           
    f, axarr = plt.subplots(nrow, ncol)

    for i in range(nclusterk):
        ilabel = np.where(labels == i)[0]
        row = int(ceil(i / 3)) - 1
        col = i % 3
        im , imf = gaia.sampling_filtering(a1[ilabel],a2[ilabel], BINSIZE, SIGMA)
        
        axarr[row,col].imshow(imf, cmap = 'gist_stern')
        axarr[row,col].set_xlabel(xylab[0])
        axarr[row,col].set_ylabel(xylab[1])
        
        txt = "Stars:%d Dist:%3.1f Vra:%3.1f Vdec:%3.1f"%(len(ilabel), centroid[i][2],centroid[i][3],centroid[i][4])
        axarr[row,col].text(-0.1,1.02, txt, size=12, ha="left", 
         transform=axarr[row,col].transAxes)
        
    plt.savefig(figname)
    plt.show()
  

def plot3d_labels(a1,a2,a3,labels,nclusterk,centroid,xylab = ["X","Y","Z"],title = "Title", clustername= "NGC"):
    figname = "%s-3Dkmeans-gaia.png"%(clustername)
    rcParams['figure.figsize'] = 14, 21
    nrow = int(ceil(nclust / 3))
    ncol = 3
    
    fig = plt.figure()

    for i in range(nclusterk):
        ilabel = np.where(labels == i)[0]
        row = int(ceil(i / 3)) 
        col = i % 3 + 1
        axarr = fig.add_subplot(nrow,ncol,i+1, projection='3d')
        axarr.scatter(a1[ilabel],a2[ilabel],a3[ilabel], c="r",marker ="*")
        axarr.set_xlabel(xylab[0])
        axarr.set_ylabel(xylab[1])
        axarr.set_zlabel(xylab[2])
    
    plt.savefig(figname)
    plt.show()

In [ ]:
## astrometric conversion
## 
def convert_to_cartesian(lgal, bga, dist):
    "Convert ra,dec (ICRS) and distance (pc) to Cartesian reference"
    
    xx = np.zeros(len(lgal))
    yy = np.zeros(len(lgal))
    zz = np.zeros(len(lgal))
    
    for i in range(len(lgal)):
        c = coord.SkyCoord(l=lgal[i]*u.degree, b=bgal[i]*u.degree, distance=dist[i]*u.pc, frame='galactic')
        
        xx[i] = c.cartesian.x.value
        yy[i] = c.cartesian.y.value
        zz[i] = c.cartesian.z.value
        
    return(xx,yy,zz)

In [ ]:
## Reading the votable

#voname = "m47_off.vot"
#voname = "ngc2232.vot"
# voname = "ngc2682_2deg.vot"
    
lgal = data['l']
bgal = data['b']
pmas = data['parallax']
distance = 1000. / np.ma.filled(pmas, -9999.)
pmra = np.ma.filled(data['pmra'], -999999999.)
pmdec= np.ma.filled(data['pmdec'],-9999999.)
vdec = 4.74 * WEIGHTVEL * pmdec / pmas   ##?
vra  = 4.74 * WEIGHTVEL * pmra  / pmas


print("## Total stars: %d"%(len(lgal)))

## mashing the data ...
##


# datask = np.zeros((len(vra),5))
datask = filter_data(lgal,bgal,distance,vra,vdec ,cartesian = False)
datanor2, normalization_vector2 = normalization0_1(datask)


print("## Stars selected: %d"%(len(datask[:,0])))


## fitted cluster in k-means
nclust = kCluster

print("## computing k-means...")
kmeans = KMeans(n_clusters=nclust, max_iter = 500, n_init = 50)
kmeans.fit(datanor2)

centroid = unnormalization0_1(kmeans.cluster_centers_,normalization_vector2)

# centroid = kmeans.cluster_centers_
labels = kmeans.labels_

print("## Centroids:")
print(centroid)

#print(labels)
for i in range(nclust):
    nstar = len(np.where(labels == i)[0])
    print("## Label: %d, %d stars"%(i,nstar))


colors = ["r.","k.","g.","y.","b.","k.","b.","g.","y.","c.","r.","k.","g.","y.","w.","k.","b.","g.","y.","c.","r.","k.","g.","y.","w.","k.","b.","g.","y.","c."]
rcParams['figure.figsize'] = 12, 12

for i in range(nclust):
    ilabel = np.where(labels == i)[0]
    plt.plot(datask[ilabel,3],datask[ilabel,4],colors[i],markersize=10)
    
plt.show()

## plot the different clusters

a1 = datask[:,0]     ## l
a2 = datask[:,1]     ## b
plot2d_labels(a1,a2,labels,nclust,centroid,xylab = ["l","b"],title = "Title", clustername= "%s-%3.0fpc"%(clustername, distclust))
plot2d_filtered_labels(a1,a2,labels,nclust,centroid,xylab = ["l","b"],title = "Title", clustername= "%s-%3.0fpc"%(clustername, distclust))



### Elbow method
X = datask
distorsions = []
for k in range(2, 20):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(X)
    distorsions.append(kmeans.inertia_)

fig = plt.figure(figsize=(15, 5))
plt.plot(range(2, 20), distorsions)
plt.grid(True)
plt.title('Elbow curve')
plt.show()

    